In [ ]:
# Check sample positions from Hermes HDF5 file

import numpy as np
import h5py as h5py 
import argparse # only for argument parsing 

POS_X_PATH_C="/entry1/camera/real_sample_x"
POS_Y_PATH_C="/entry1/camera/real_sample_y"


def _open_dataset_maybe_group(f: h5py.File, path: str) -> np.ndarray:
    """
    Return a numpy array from either a dataset at `path` or, if `path` is a group,
    a child dataset named 'data'. Raises KeyError if not found.
    """
    obj = f[path]
    if isinstance(obj, h5py.Dataset):
        return obj[()]
    if isinstance(obj, h5py.Group):
        # common NeXus convention uses a dataset called 'data' inside the group
        for candidate in ("data", "value", "values"):
            if candidate in obj and isinstance(obj[candidate], h5py.Dataset):
                return obj[candidate][()]
    raise KeyError(f"No dataset found at or under {path!r} (looked for 'data').")


with h5py.File('/Users/gvr/code/hermes_ptypy_ingest/beamline_ptypy_ingest/data/Image_20250924_025/Image_20250924_010.hdf5', "r") as fC:
    x = _open_dataset_maybe_group(fC, POS_X_PATH_C)
    y = _open_dataset_maybe_group(fC, POS_Y_PATH_C)
if x.shape != y.shape:
    raise ValueError(f"Position arrays shapes differ: {x.shape} vs {y.shape}")
# Flatten into a list of (x,y) in requested scan order
order = "row"  # "row" or "col"
if order not in {"row", "col"}:
    raise ValueError("--order must be 'row' or 'col'")
if order == "row":
    xf = np.ravel(x, order="C")
    yf = np.ravel(y, order="C")
else:
    xf = np.ravel(x, order="F")
    yf = np.ravel(y, order="F")
pos = np.stack([xf, yf], axis=1).astype(np.float64)


import matplotlib.pyplot as plt
plt.scatter(xf, yf, s=5); 
plt.gca().set_aspect('equal')
plt.title("Real Sample  positions (x,y)"); plt.xlabel("x (m)"); plt.ylabel("y (m)")
plt.show()  # show scan positions
